# Theano

In [1]:
# data import
import scipy.io
ratdata = scipy.io.loadmat('chrono_B069_rawdata.mat')

In [2]:
# trial index starts from 0 

def trialdata(ratdata,trial):
    if ratdata['rawdata']['pokedR'][0][trial] > 0 :
        rat_choice = 1 # "R"
    else : 
        rat_choice = -1 # "L"
        
    return np.require(ratdata['rawdata']['rightbups'][0][trial][0],requirements='A'), \
np.require(ratdata['rawdata']['leftbups'][0][trial][0],requirements='A'), \
ratdata['rawdata']['T'][0][trial][0][0], rat_choice


In [3]:
import numpy as np
import theano
import theano.tensor as T

In [4]:
# trial index starts from 0 
RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, 0) 
print LeftClickTimes
print rat_choice

[ 0.       0.19235  0.34361]
-1


In [5]:
# Global variables 
epsilon = 10.0**(-10) 
dx = 0.25
dt = 0.02

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

## bin_centers = make_bins(B, dx, binN)

In [129]:
#### make_bins (Theano version)
def make_bins(B, dx, binN):
    bins = (T.arange(binN)+1)*B
    bins = dx*bins/B

    tmp = T.scalar()
    
    bins = T.switch(T.eq(bins[-1],B),
                    T.set_subtensor(bins[-1], B+dx),
                    T.set_subtensor(bins[-1], 2*B - bins[-2]))
    
    bins = T.concatenate((-bins[::-1], T.zeros(1), bins))
    return bins

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [ ]:
def Fmatrix(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
    F = np.zeros((bin_centers.size,bin_centers.size))
    
    if lam == 0:  # for Fi
        mus = bin_centers*np.exp(lam*dt)
    else:        
        mus = np.exp(lam*dt)*(bin_centers + c/lam) - c/lam

        
# modified it according to gaussbins.m -amyoon
#     sbinsize = 0.1*sigma_sbin
#     swidth = 4*sigma_sbin
#     sbins = np.arange(-swidth,swidth+epsilon,sbinsize)
    n_sbins = max(70, np.ceil(10*np.sqrt(sigma2_sbin)/dx))

    swidth = 5*np.sqrt(sigma2_sbin)
    sbins = np.linspace(-swidth,swidth,n_sbins*2+1)
    sbinsize = sbins[1]-sbins[0]

    ps = np.exp(-sbins**2/(2*sigma2)) 
    ps = ps/sum(ps)
            
    base_sbins = sbins
        
    for j in np.arange(bin_centers.size-1)+1:
        sbins = base_sbins + mus[j]
        tmp = 1
        
        for k in range(sbins.size):
            if sbins[k] <= bin_centers[0]:
                F[0,j] += ps[k]
            elif sbins[k] >= bin_centers[-1]:
                F[-1,j] += ps[k]
            else:
                if (sbins[k] > bin_centers[0] and sbins[k] < bin_centers[1]):
                    bottom = 0; top = 1;
                elif (sbins[k] > bin_centers[-2] and sbins[k] < bin_centers[-1]):
                    bottom = bin_centers.size-2; top = bin_centers.size-1;
                else :    
                    bottom = int(np.floor((sbins[k]-bin_centers[1])/dx) + 1);
                    top = int(np.ceil((sbins[k]-bin_centers[1])/dx) + 1);

                if bottom < 0: bottom = 0; 
                if top < 0: top = 0; 

                if bottom == top:
                    F[bottom,j] += ps[k]
                else:
                    F[top,j] += ps[k]*(sbins[k] - bin_centers[bottom])/(bin_centers[top] - bin_centers[bottom])
                    F[bottom,j] += ps[k]*(bin_centers[top] - sbins[k])/(bin_centers[top] - bin_centers[bottom])
                        
    F[:,0] = 0
    F[:,-1] = 0
    F[0,0] = 1
    F[-1,-1] = 1
    
    return F

## LL = logProbRight(params)

params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]

In [7]:

# =========================== Model ================================ #

def logLike(sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, RightClickTimes, LeftClickTimes, maxT, rat_choice):
    tf_params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]
    if T.gt(rat_choice , 0): # == "R":
        LP_right = logProbRight(tf_params, RightClickTimes, LeftClickTimes, maxT)
#         print "R : " + str(LP_right)
        return LP_right
    elif T.lt(rat_choice , 0): # == "L":
        LP_right = logProbRight(tf_params, RightClickTimes, LeftClickTimes, maxT)
#         print "L : " + str(np.log(1 - np.exp(LP_right)))
        return np.log(1 - np.exp(LP_right))
    else:
        print -1. # "Rat did what?? It was neither R nor L"
        
def logProbRight(params, RightClickTimes, LeftClickTimes, maxT):
#     sigma_a = params['sigma_a']
#     sigma_s = params['sigma_s']
#     sigma_i = params['sigma_i']
#     lam = params['lambda']
#     B = params['B']
#     bias = params['bias']
#     phi = params['phi']
#     tau_phi = params['tau_phi']
#     lapse = params['lapse']
#     params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]  
    
    sigma2_a = params[0]
    sigma2_s = params[1]
    sigma2_i = params[2]
    lam = params[3]
    B = params[4]
    bias = params[5]
    phi = params[6]
    tau_phi = params[7]
    lapse = params[8]
    
    global a, dx, dt, a_trace, c_trace
        
    Nsteps = T.cast(T.ceil(maxT/dt),'int32')

#     if RightClickTimes.size == 0:
#         RightClickTimes = np.empty(0)
#     if LeftClickTimes.size == 0:
#         LeftClickTimes = np.empty(0)

    print Nsteps
    
#     net_input = tf.zeros([1, Nsteps.eval()], tf.float64)
#     total_input = tf.zeros([1, Nsteps.eval()], tf.float64)
#     NClicks = tf.zeros([1, Nsteps], tf.int32)    
#     net_input = tf.placeholder(tf.float64, [1, None],name='net_input')
#     total_input = tf.placeholder(tf.float64, [1, None],name='total_input')
#     NClicks = tf.placeholder(tf.int32, [1, None], name='Nclicks')    

        
    # ====== Make adapted clicks
    
    Lsame = T.ones(T.shape(LeftClickTimes))
    Rsame = T.ones(T.shape(RightClickTimes))
    # magnitude of stereo clicks set to zero   
    Lsame = T.set_subtensor(Lsame[0],0)
    Rsame = T.set_subtensor(Rsame[0],0)
    
    # inter-click-intervals
    ici_L = T.extra_ops.diff(LeftClickTimes)
    ici_R = T.extra_ops.diff(RightClickTimes)

    
    for i in np.arange(np.size(LeftClickTimes)-1)+1:
        last_L = tau_phi*np.log(1-Lsame[0][i-1]*phi)
        Lsame[0][i] = 1 - np.exp((-ici_L[i-1] + last_L)/tau_phi)
        
    for i in np.arange(np.size(RightClickTimes)-1)+1:
        last_R = tau_phi*np.log(1-Rsame[0][i-1]*phi)
        Rsame[0][i] = 1 - np.exp((-ici_R[i-1] + last_R)/tau_phi)

    Lsame = Lsame.real    
    Rsame = Rsame.real   
        
    # index starts from 0
    # net_input / total_input
    # Counting number of clicks
    cnt = 0
    for i in np.ceil((RightClickTimes+epsilon)/dt).astype(int):
        NClicks[0][i-1] += 1
        net_input[0][i-1] += Rsame[0][cnt]
        total_input[0][i-1] += Rsame[0][cnt]
        cnt += 1
    
    cnt = 0 
    for i in np.ceil((LeftClickTimes+epsilon)/dt).astype(int):
        NClicks[0][i-1] += 1  
        net_input[0][i-1] -= Lsame[0][cnt]
        total_input[0][i-1] += Lsame[0][cnt]
        cnt += 1        
       
    binN = int(np.ceil(B/dx))
    
    bin_centers = make_bins(B,dx,binN)
        
    a_trace = np.zeros((bin_centers.size, Nsteps))
    c_trace = np.zeros((1, Nsteps+1))
    a0 = np.zeros(bin_centers.size)
    a0[binN] = 1-2*lapse; a0[0] = lapse; a0[-1] = lapse;
    
    Fi = Fmatrix([sigma2_i, 0, 0.0], bin_centers)
    
    a = np.matmul(Fi,a0)
    a_trace[:,0] = a.flatten()
 
    F0 = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    for i in np.arange(Nsteps-1)+1:
        if total_input[0][i-1]==0:
            a = np.matmul(F0,a)
        else:
            total_var = sigma2_a*dt + (sigma2_s*total_input[0][i-1])/40
            F = Fmatrix([total_var, lam, net_input[0][i-1]/dt], bin_centers)
            a = np.matmul(F,a)
            
        c_trace[0][i] = net_input[0][i-1]
        a_trace[:,i] = a.flatten()

    bias_bottom = int(np.floor((bias-bin_centers[1])/dx) + 1)
    bias_top = int(np.ceil((bias-bin_centers[1])/dx) + 1) # top
    
    Pd = np.zeros((1, bin_centers.size))
    Pd[0][-bias_top+1:] = a[-bias_top+1:]
    if bias_bottom == bias_top:
        Pd[0][bias_bottom] = a[bias_bottom]/2
    else:
        dh = bin_centers[bias_top] - bias
        dl = bias - bin_centers[bias_bottom]
        dd = dh + dl
        Pd[0][bias_top] = a[bias_top]*(0.5 + dh/dd/2)
        Pd[0][bias_bottom] = a[bias_bottom]*(dh/dd/2)
    
    pright = sum(Pd[0])
    
    return np.log(pright)


# =========== Declare Theano symbolic variables =========== #

## Variables 
# 1. (can assign the name of each variable later..)
# tf_params = tf.Variable(params, name = "params")

# 2. 
# tf_params = {
#     'sigma_a' : tf.Variable(sigma_a, name="sigma_a"),
#     'sigma_s' : tf.Variable(sigma_s, name="sigma_s"),
#     'sigma_i' : tf.Variable(sigma_i, name="sigma_i"),
#     'lamda' : tf.Variable(lam, name="lambda"),
#     'B' : tf.Variable(B, name="B"),
#     'bias' : tf.Variable(bias, name="bias"),
#     'phi' : tf.Variable(phi, name="phi"),
#     'tau_phi' : tf.Variable(tau_phi, name="tau_phi"),
#     'lapse' : tf.Variable(lapse, name="lapse")
# }

# 3.
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;

sigma_a = theano.shared(sigma_a, name="sigma_a")
sigma_s = theano.shared(sigma_s, name="sigma_s")
sigma_i = theano.shared(sigma_i, name="sigma_i")
lam = theano.shared(lam, name="lambda")
B = theano.shared(B, name="B")
bias = theano.shared(bias, name="bias")
phi = theano.shared(phi, name="phi")
tau_phi = theano.shared(tau_phi, name="tau_phi")
lapse = theano.shared(lapse, name="lapse")
# tf_params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

# ----------tip
# values = [1.0, 3.0]
# vars_ = [tf.Variable([v], dtype=dtype) for v in values]


# =========== Declare Theano symbolic variables =========== #
## inputs
RightClickTimes = T.dvector()#tf.placeholder(tf.float32, name = 'RightClickTimes') # Right Clicks
LeftClickTimes = T.dvector()#tf.placeholder(tf.float32, name = 'LeftClickTimes') # Left Clicks
maxT = T.dscalar()#tf.placeholder(tf.float32, name = 'maxT')
rat_choice = T.wscalar()#tf.placeholder(tf.float32, name = 'rat_choice')


# ==================== Construct Model ========================= #

# LL = tf.py_func(logLike, [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, 
#                           RightClickTimes, LeftClickTimes, maxT, rat_choice], [tf.float64])


LL = logLike(sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, 
                          RightClickTimes, LeftClickTimes, maxT, rat_choice)

train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, maxT, rat_choice],
#     updates = 
    outputs = [LL]
)


# opt = tf.train.AdamOptimizer(0.1)
# grads_and_vars = opt.compute_gradients(tf.to_float(LL[0]),[sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse])
# grads2 = [g for g, _ in grads_and_vars]
# vars2 = [v for _, v in grads_and_vars]

# var_grad = tf.gradients(LL[0],[sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse])[0]

# ==================== Launch the Graph ========================= #

# init = tf.initialize_all_variables()
# with tf.Session() as sess:
#     sess.run(init)
    
total_LL = 0.

# Training cycle
for i in range(10):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)

    LLi = train(RClickTimes1, LClickTimes1, maxT1, rat_choice1)

    total_LL += LLi


    print "Trial :", '%04d' % (i+1), "LL = ", "{:.9f}".format(total_LL)

print "Done."



Elemwise{Cast{int32}}.0


TypeError: a float is required

In [8]:
a = T.dvector()
out = T.extra_ops.diff(a)
f = theano.function([a], out)
print(f([0, 1, 2]))

[ 1.  1.]


In [10]:
t = theano.tensor.arange(9).reshape((3,3))
t2=T.set_subtensor(t[0],1)
a = t2[0].eval()
a[1]

1

### Testing!!!!

In [299]:
def trialdata(ratdata,trial):
    if ratdata['rawdata']['pokedR'][0][trial] > 0 :
        rat_choice = 1 # "R"
    else : 
        rat_choice = -1 # "L"
        
    return np.require(ratdata['rawdata']['rightbups'][0][trial][0],requirements='A'), \
np.require(ratdata['rawdata']['leftbups'][0][trial][0],requirements='A'), \
ratdata['rawdata']['T'][0][trial][0][0], rat_choice

def practice_LL(sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, 
                          RightClickTimes, LeftClickTimes, Time_bins, rat_choice):
    
    pright = T.dscalar()
    
    # ==== inter-click-intervals and Make adapted clicks
    
    net_input = T.vector("net_input")
    total_input = T.vector("total_input")


    # :: can be integrated with one function
    
    # sequences: T.extra_ops.diff(LeftClickTimes) // inter-click-intervals
    # prior results: lsame (0) // initial value
    # non-sequences: phi, tau_phi // parameters
    Lsame, updates = theano.scan(lambda ici, lsame, phi, tau_phi :
                                     1-T.exp((tau_phi*T.log(1-lsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(LeftClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )
    Rsame, updates = theano.scan(lambda ici, rsame, phi, tau_phi : 
                                 1-T.exp((tau_phi*T.log(1-rsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(RightClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )

    Lsame = T.concatenate([T.zeros(1, dtype=Lsame.dtype), Lsame[:,0]],axis=0)
    Rsame = T.concatenate([T.zeros(1, dtype=Rsame.dtype), Rsame[:,0]],axis=0)

    L_here =  T.cast(T.floor((LeftClickTimes+epsilon)/dt),'int32')
    R_here =  T.cast(T.floor((RightClickTimes+epsilon)/dt),'int32')

    dtype=theano.config.floatX

    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=dtype)

    template = T.vector('template')

    ## ==== Collect the adapted click values
    # index starts from 0
    # net_input / total_input

    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    net_input_l, updates = theano.scan(lambda lhere,lsame, tmp :
                                     T.set_subtensor(tmp[lhere],tmp[lhere]+lsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [L_here, Lsame]
                                  )
    net_input_r, updates = theano.scan(lambda rhere,rsame, tmp :
                                     T.set_subtensor(tmp[rhere],tmp[rhere]+rsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [R_here, Rsame]
                                  )

    net_input = net_input_r[-1] - net_input_l[-1]
    total_input = net_input_r[-1] + net_input_l[-1]
    
    ## ==== make bins
    binN = T.cast(T.ceil(B/dx),'int32')
    bin_centers = make_bins(B,dx,binN)
        
    ## ==== make P init
    a0 = T.zeros_like(bin_centers)
    a0 = T.set_subtensor(a0[binN],1-2*lapse)
    a0 = T.set_subtensor(a0[0],lapse)
    a0 = T.set_subtensor(a0[-1],lapse)
    
#     a_trace = np.zeros((bin_centers.size, Nsteps))
#     c_trace = np.zeros((1, Nsteps+1))

    ## ==== Fi
    Fi = Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
    
    a = T.dot(Fi,a0)
#     a_trace[:,0] = a.flatten()
 
    F0 = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    
    
    def a_tracing(tot_input,net_input,a,F0,bin_centers,sigma2_a,sigma2_s,lam):
        total_var = sigma2_a*dt + (sigma2_s*tot_input)/40
        F = Fmatrix([total_var, lam, net_input/dt], bin_centers)
        a_rest = T.dot(F,a)
        
        a = T.switch(T.eq(tot_input,0),
                     T.dot(F0,a),
                     a_rest
                     )
        
        return a
        
    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    res, _ = theano.scan(fn = a_tracing,
                             outputs_info = a,
                             sequences = [total_input, net_input],
                             non_sequences = [F0,bin_centers,sigma2_a,sigma2_s,lam]
                        )

    
#     Pd = np.zeros((1, bin_centers.size))
#     Pd[0][-bias_top+1:] = a[-bias_top+1:]
#     if bias_bottom == bias_top:
#         Pd[0][bias_bottom] = a[bias_bottom]/2
#     else:
#         dh = bin_centers[bias_top] - bias
#         dl = bias - bin_centers[bias_bottom]
#         dd = dh + dl
#         Pd[0][bias_top] = a[bias_top]*(0.5 + dh/dd/2)
#         Pd[0][bias_bottom] = a[bias_bottom]*(dh/dd/2)
    
#     pright = sum(Pd[0])
    
    pright = rat_choice + a[0] + res[0][0] + net_input[0] + a0[binN] + Fi[0][0] + F0[0][0] 
    return pright
    
# 3.
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;

sigma2_a = theano.shared(sigma_a, name="sigma_a")
sigma2_s = theano.shared(sigma_s, name="sigma_s")
sigma2_i = theano.shared(sigma_i, name="sigma_i")
lam = theano.shared(lam, name="lambda")
B = theano.shared(B, name="B")
bias = theano.shared(bias, name="bias")
phi = theano.shared(phi, name="phi")
tau_phi = theano.shared(tau_phi, name="tau_phi")
lapse = theano.shared(lapse, name="lapse")
    
# =========== Declare Theano symbolic variables =========== #
## inputs
RightClickTimes = T.dvector("RightClickTimes") # Right Clicks
LeftClickTimes = T.dvector("LeftClickTimes") # Left Clicks
Time_bins = T.vector("Time_bins") # Time_bins
rat_choice = T.wscalar("rat_choice") # rat_choice

# ==================== Construct Model ========================= #

# LL = tf.py_func(logLike, [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, 
#                           RightClickTimes, LeftClickTimes, maxT, rat_choice], [tf.float64])


LL = practice_LL(sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, 
                          RightClickTimes, LeftClickTimes, Time_bins, rat_choice)

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = [LL]
)

    
total_LL = 0.

# Training cycle
for i in range(2):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=dtype)

    LLi = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

    total_LL += LLi[0]


    print "Trial :", '%04d' % (i+1), "LL = ", "{:.9f}".format(total_LL)

print "Done."



Join.0
Trial : 0001 LL =  2.211745769
Trial : 0002 LL =  6.211745769
Done.


In [293]:
RightClickTimes = T.dvector("RightClickTimes")# Right Clicks
LeftClickTimes = T.dvector("LeftClickTimes")# Left Clicks
template = T.vector('template')
rat_choice = T.wscalar("rat_choice")

# ==== inter-click-intervals and Make adapted clicks
net_input = T.vector("net_input")
total_input = T.vector("total_input")

# sequences: T.extra_ops.diff(LeftClickTimes) // inter-click-intervals
# prior results: lsame (0) // initial value
# non-sequences: phi, tau_phi // parameters
Lsame, updates = theano.scan(lambda ici, lsame, phi, tau_phi :
                                 1-T.exp((tau_phi*T.log(1-lsame*phi)-ici)/tau_phi),
                              outputs_info = T.zeros(1),
                              sequences=[T.extra_ops.diff(LeftClickTimes)],
                              non_sequences=[phi, tau_phi]
                              )
Rsame, updates = theano.scan(lambda ici, rsame, phi, tau_phi : 
                             1-T.exp((tau_phi*T.log(1-rsame*phi)-ici)/tau_phi),
                              outputs_info = T.zeros(1),
                              sequences=[T.extra_ops.diff(RightClickTimes)],
                              non_sequences=[phi, tau_phi]
                              )

Lsame = T.concatenate([T.zeros(1, dtype=Lsame.dtype), Lsame[:,0]],axis=0)
Rsame = T.concatenate([T.zeros(1, dtype=Rsame.dtype), Rsame[:,0]],axis=0)

L_here =  T.cast(T.floor((LeftClickTimes+epsilon)/dt),'int32')
R_here =  T.cast(T.floor((RightClickTimes+epsilon)/dt),'int32')

# ==== Collect the adapted click values
# index starts from 0
# net_input / total_input

# sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
# prior results: zeros // initial value
# non-sequences: Lsame // parameters
net_input_l, updates = theano.scan(lambda lhere,lsame, tmp :
                                 T.set_subtensor(tmp[lhere],tmp[lhere]+lsame),
                              outputs_info = T.zeros_like(template),
                              sequences = [L_here, Lsame]
                              )
net_input_r, updates = theano.scan(lambda rhere,rsame, tmp :
                                 T.set_subtensor(tmp[rhere],tmp[rhere]+rsame),
                              outputs_info = T.zeros_like(template),
                              sequences = [R_here, Rsame]
                              )

net_input = net_input_r[-1] - net_input_l[-1]
total_input = net_input_r[-1] + net_input_l[-1]

test_func = theano.function(
    inputs=[LeftClickTimes,RightClickTimes,template],
    outputs=[net_input,total_input]
)


############ testing with REAL data 
RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, 0)

dtype=theano.config.floatX

np_Nstep = int(np.ceil(maxT1/dt))
np_empty_tmp = np.zeros((np_Nstep), dtype=dtype)



test_func(LClickTimes1,RClickTimes1,np_empty_tmp)


[array([ 0.21174577,  0.        ,  0.        ,  0.        ,  0.        ,
         0.59572769,  0.        ,  0.        ,  0.        , -0.85390527,
         0.68879055,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.0679164 , -0.83610755]),
 array([ 0.21174577,  0.        ,  0.        ,  0.        ,  0.        ,
         0.59572769,  0.        ,  0.        ,  0.        ,  0.85390527,
         0.68879055,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.0679164 ,  0.83610755])]

In [95]:
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;

sigma_a = theano.shared(sigma_a, name="sigma_a")
sigma_s = theano.shared(sigma_s, name="sigma_s")
sigma_i = theano.shared(sigma_i, name="sigma_i")
lam = theano.shared(lam, name="lambda")
B = theano.shared(B, name="B")
bias = theano.shared(bias, name="bias")
phi = theano.shared(phi, name="phi")
tau_phi = theano.shared(tau_phi, name="tau_phi")
lapse = theano.shared(lapse, name="lapse")



sigma_a


In [131]:
#### make_bins (Theano version)
def make_bins(B, dx, binN):
    bins = (T.arange(binN)+1)*B
    bins = dx*bins/B

    tmp = T.scalar()
    
    bins = T.switch(T.eq(bins[-1],B),
                    T.set_subtensor(bins[-1], B+dx),
                    T.set_subtensor(bins[-1], 2*B - bins[-2]))
    
    bins = T.concatenate((-bins[::-1], T.zeros(1), bins))
    return bins

binN = T.cast(T.ceil(B/dx),'int32')
bin_centers = make_bins(B, dx, binN)

test_func = theano.function(
    inputs=[],
    outputs=[bin_centers, binN, test]
)

test_func()

[array([-4.2 , -4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25,
        -2.  , -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,
         0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,
         2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.2 ]),
 array(17, dtype=int32),
 array(-0.09999999999999964)]

In [288]:
#### Fmatrix (Theano version)
def Fmatrix_i(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
    F = T.zeros((bin_centers.shape[0],bin_centers.shape[0]),'float32')
    
    # for Fi (when lambda = 0)
    mus = bin_centers*T.exp(lam*dt)

#     mus = T.exp(lam*dt)*(bin_centers + c/lam) - c/lam
        
    tmp = T.ceil(10*T.sqrt(sigma2_sbin)/dx)
    n_sbins = T.switch(T.ge(tmp,70),tmp,70)
    
    swidth = 5*T.sqrt(sigma2_sbin)
    sbins = (T.arange(n_sbins)+1)/n_sbins*swidth#np.linspace(-swidth,swidth,n_sbins*2+1)
    sbins = T.concatenate((-sbins[::-1], T.zeros(1), sbins))

    ps = T.exp(-sbins**2/(2*sigma2)) 
    ps = ps/T.sum(ps)
            
    base_sbins = sbins
        
        
    def inner_loop(sbin, p, hp, lp, F, bin_centers,j):
        dd = bin_centers[hp] - bin_centers[lp]
        
        FF = T.set_subtensor(F[hp,j], F[hp,j]+p*(sbin-bin_centers[lp])/dd)
        FF = T.set_subtensor(FF[lp,j], FF[lp,j]+p*(bin_centers[hp]-sbin)/dd)
        
        F_rest = T.switch(T.eq(dd,0),
                          T.set_subtensor(F[lp,j], F[lp,j]+p),
                          FF)
        
        F = T.switch(T.le(sbin,bin_centers[0]),
                          T.set_subtensor(F[0,j], F[0,j]+p),
                          F)
        F = T.switch(T.ge(sbin,bin_centers[-1]),
                          T.set_subtensor(F[-1,j], F[-1,j]+p),
                          F)
        F = T.switch(T.gt(sbin,bin_centers[0]) & T.lt(sbin,bin_centers[-1]),
                          F_rest,#T.set_subtensor(F[-1,j], -3),#F_rest,
                          F)

        
        return F
    
    def outer_loop(mu,j, F, base_sbins, ps, bin_centers):
        sbins = base_sbins + mu
        
        n = bin_centers.shape[0]-1
        
        hps = T.cast(T.ceil( (sbins-bin_centers[1])/dx) +1,'int32')
        lps = T.cast(T.floor((sbins-bin_centers[1])/dx) +1,'int32')
        
        hps = T.set_subtensor(hps[(hps < 0).nonzero()],0)
        lps = T.set_subtensor(lps[(lps < 0).nonzero()],0)

        hps = T.set_subtensor(hps[(hps > n).nonzero()],n)
        lps = T.set_subtensor(lps[(lps > n).nonzero()],n)

        hps = T.set_subtensor(hps[T.and_(bin_centers[0]<sbins, sbins<bin_centers[1]).nonzero()],1)
        lps = T.set_subtensor(lps[T.and_(bin_centers[-2]<sbins, sbins<bin_centers[-1]).nonzero()],n-1)

        hps = T.set_subtensor(hps[(bin_centers[-1]<sbins).nonzero()],n)
        lps = T.set_subtensor(lps[(bin_centers[-1]<sbins).nonzero()],n)

        hps = T.set_subtensor(hps[(sbins<bin_centers[0]).nonzero()],0)
        lps = T.set_subtensor(lps[(sbins<bin_centers[0]).nonzero()],0)

        # sequences: sbins, ps, hps, lps // 
        # prior results: F //
        # non-sequences: bin_centers, index j // 
        results,_ = theano.scan(fn=inner_loop,
                               outputs_info = F,
                               sequences = [sbins, ps, hps, lps],
                               non_sequences = [bin_centers,j]
                               )
        F = results[-1]
        
        return F
        
    # sequences: mus[1:], array(1:binsize) // mus, indices of the outer loop
    # prior results: zeros(size(F)) // initialize F as zeros
    # non-sequences: base_bins, ps, bin_centers // 
    res,_ = theano.scan(fn = outer_loop,
                      outputs_info = T.zeros_like(F),
                      sequences = [mus[1:], T.arange(bin_centers.shape[0]-1)+1],
                      non_sequences = [base_sbins, ps, bin_centers]
                     )
    F = res[-1]
    
    F = T.set_subtensor(F[:,0], 0)
    F = T.set_subtensor(F[:,-1], 0)
    F = T.set_subtensor(F[0,0], 1)
    F = T.set_subtensor(F[-1,-1], 1)
    
    return F

In [290]:
#### Fmatrix (Theano version)
def Fmatrix(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
    F = T.zeros((bin_centers.shape[0],bin_centers.shape[0]),'float32')
    
#     mus = T.switch(T.eq(lam,0),bin_centers*T.exp(lam*dt),T.exp(lam*dt)*(bin_centers + c/lam) - c/lam)
    mus = T.exp(lam*dt)*(bin_centers + c/lam) - c/lam
        
    tmp = T.ceil(10*T.sqrt(sigma2_sbin)/dx)
    n_sbins = T.switch(T.ge(tmp,70),tmp,70)
    
    swidth = 5*T.sqrt(sigma2_sbin)
    sbins = (T.arange(n_sbins)+1)/n_sbins*swidth#np.linspace(-swidth,swidth,n_sbins*2+1)
    sbins = T.concatenate((-sbins[::-1], T.zeros(1), sbins))

    ps = T.exp(-sbins**2/(2*sigma2)) 
    ps = ps/T.sum(ps)
            
    base_sbins = sbins
        
        
    def inner_loop(sbin, p, hp, lp, F, bin_centers,j):
        dd = bin_centers[hp] - bin_centers[lp]
        
        FF = T.set_subtensor(F[hp,j], F[hp,j]+p*(sbin-bin_centers[lp])/dd)
        FF = T.set_subtensor(FF[lp,j], FF[lp,j]+p*(bin_centers[hp]-sbin)/dd)
        
        F_rest = T.switch(T.eq(dd,0),
                          T.set_subtensor(F[lp,j], F[lp,j]+p),
                          FF)
        
        F = T.switch(T.le(sbin,bin_centers[0]),
                          T.set_subtensor(F[0,j], F[0,j]+p),
                          F)
        F = T.switch(T.ge(sbin,bin_centers[-1]),
                          T.set_subtensor(F[-1,j], F[-1,j]+p),
                          F)
        F = T.switch(T.gt(sbin,bin_centers[0]) & T.lt(sbin,bin_centers[-1]),
                          F_rest,#T.set_subtensor(F[-1,j], -3),#F_rest,
                          F)

        
        return F
    
    def outer_loop(mu,j, F, base_sbins, ps, bin_centers):
        sbins = base_sbins + mu
        
        n = bin_centers.shape[0]-1
        
        hps = T.cast(T.ceil( (sbins-bin_centers[1])/dx) +1,'int32')
        lps = T.cast(T.floor((sbins-bin_centers[1])/dx) +1,'int32')
        
        hps = T.set_subtensor(hps[(hps < 0).nonzero()],0)
        lps = T.set_subtensor(lps[(lps < 0).nonzero()],0)

        hps = T.set_subtensor(hps[(hps > n).nonzero()],n)
        lps = T.set_subtensor(lps[(lps > n).nonzero()],n)

        hps = T.set_subtensor(hps[T.and_(bin_centers[0]<sbins, sbins<bin_centers[1]).nonzero()],1)
        lps = T.set_subtensor(lps[T.and_(bin_centers[-2]<sbins, sbins<bin_centers[-1]).nonzero()],n-1)

        hps = T.set_subtensor(hps[(bin_centers[-1]<sbins).nonzero()],n)
        lps = T.set_subtensor(lps[(bin_centers[-1]<sbins).nonzero()],n)

        hps = T.set_subtensor(hps[(sbins<bin_centers[0]).nonzero()],0)
        lps = T.set_subtensor(lps[(sbins<bin_centers[0]).nonzero()],0)

        # sequences: sbins, ps, hps, lps // 
        # prior results: F //
        # non-sequences: bin_centers, index j // 
        results,_ = theano.scan(fn=inner_loop,
                               outputs_info = F,
                               sequences = [sbins, ps, hps, lps],
                               non_sequences = [bin_centers,j]
                               )
        F = results[-1]
        
        return F
        
    # sequences: mus[1:], array(1:binsize) // mus, indices of the outer loop
    # prior results: zeros(size(F)) // initialize F as zeros
    # non-sequences: base_bins, ps, bin_centers // 
    res,_ = theano.scan(fn = outer_loop,
                      outputs_info = T.zeros_like(F),
                      sequences = [mus[1:], T.arange(bin_centers.shape[0]-1)+1],
                      non_sequences = [base_sbins, ps, bin_centers]
                     )
    F = res[-1]
    
    F = T.set_subtensor(F[:,0], 0)
    F = T.set_subtensor(F[:,-1], 0)
    F = T.set_subtensor(F[0,0], 1)
    F = T.set_subtensor(F[-1,-1], 1)
    
    return F

In [289]:
binN = T.cast(T.ceil(B/dx),'int32')
bin_centers = make_bins(B, dx, binN)
n = bin_centers.shape[0]
F = T.zeros((n,n),'float32')

Fi= Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
 

test_func = theano.function(
    inputs=[],
    outputs=Fi
)

test_func()

array([[ 1.        ,  0.41181999,  0.218907  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.19714171,  0.1721313 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.18722765,  0.21792339, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.21792339,
         0.18722765,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.1721313 ,
         0.19714172,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.21890698,
         0.41181993,  1.        ]], dtype=float32)

In [112]:
r = T.scalar('r') 
s = T.scalar('s') 
t = T.scalar('t') 
r_ = theano.printing.Print('r')(r) 
s_ = theano.printing.Print('s')(s) 


u = T.switch(T.eq(t, 0), s_, r_) 
u.eval({t: 0, r: 1, s: 2}) 

#  u.eval({t: -1, r: 1, s: 2}) 
# s __str__ = 2.0 
# r __str__ = 1.0 
 

s __str__ = 2.0
r __str__ = 1.0


array(2.0)

In [256]:
print LClickTimes1
print LClickTimes1[1:]

[ 0.        0.058855  0.22679   0.30516 ]
[ 0.058855  0.22679   0.30516 ]


In [ ]:
rng = np.random

N = 10                             # training sample size
feats = 20                              # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
D1 = rng.randint(size=N, low=0, high=2)
D2 = rng.randint(size=N+10, low=0, high=2)

In [ ]:
np.shape(D[1])

In [ ]:
import numpy

coefficients = theano.tensor.vector("coefficients")
x = T.scalar("x")

max_coefficients_supported = 10000

# Generate the components of the polynomial
components, updates = theano.scan(fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),
                                  outputs_info=None,
                                  sequences=[coefficients, theano.tensor.arange(max_coefficients_supported)],
                                  non_sequences=x)
# Sum them up
polynomial = components.sum()

# Compile a function
calculate_polynomial = theano.function(inputs=[coefficients, x], outputs=polynomial)

# Test
test_coefficients = numpy.asarray([2, 1], dtype=numpy.float32)
test_value = 3
print(calculate_polynomial(test_coefficients, test_value))

In [ ]:
import theano
import theano.tensor as T
import numpy as np

# define tensor variables
X = T.vector("X")
W = T.matrix("W")
b_sym = T.vector("b_sym")
U = T.matrix("U")
Y = T.matrix("Y")
V = T.matrix("V")
P = T.matrix("P")

results, updates = theano.scan(lambda y, p, x_tm1: T.tanh(T.dot(x_tm1, W) + T.dot(y, U) + T.dot(p, V)),
          sequences=[Y, P[::-1]], outputs_info=[X])
compute_seq = theano.function(inputs=[X, W, Y, U, P, V], outputs=results)

# test values
x = np.zeros((2), dtype=theano.config.floatX)
x[1] = 1
w = np.ones((2, 2), dtype=theano.config.floatX)
y = np.ones((5, 2), dtype=theano.config.floatX)
y[0, :] = -3
u = np.ones((2, 2), dtype=theano.config.floatX)
p = np.ones((5, 2), dtype=theano.config.floatX)
p[0, :] = 3
v = np.ones((2, 2), dtype=theano.config.floatX)

print(compute_seq(x, w, y, u, p, v))

# comparison with numpy
x_res = np.zeros((5, 2), dtype=theano.config.floatX)
x_res[0] = np.tanh(x.dot(w) + y[0].dot(u) + p[4].dot(v))
for i in range(1, 5):
    x_res[i] = np.tanh(x_res[i - 1].dot(w) + y[i].dot(u) + p[4-i].dot(v))
print(x_res)

In [ ]:
import theano
import theano.tensor as T

k = T.iscalar("k")
init = T.scalar('init')
A = T.vector("A")

# Symbolic description of the result
result, updates = theano.scan(fn=lambda prior_result, A: prior_result + A,
                              outputs_info=[init],
                              sequences=[A])

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = theano.function(inputs=[A,init], outputs=result, updates=updates)

print(power([1,2,3],0))

## Good Example (Structure)

In [ ]:
import numpy as np
import theano
import theano.tensor as T

dtype=theano.config.floatX

In [ ]:
# import reberGrammar
n_in  = 7 
n_hid = 10
n_out = 7

In [ ]:
#first dimension is time
v = T.matrix(dtype=dtype) 

In [ ]:
def rescale_weights(values, factor=1.):
    factor = np.cast[dtype](factor)
    _,svs,_ = np.linalg.svd(values)
    #svs[0] is the largest singular value                      
    values = values / svs[0]
    return values

def sample_weights(sizeX, sizeY):
    values = np.ndarray([sizeX, sizeY], dtype=dtype)
    for dx in xrange(sizeX):
        vals = np.random.uniform(low=-1., high=1.,  size=(sizeY,))
        #vals_norm = np.sqrt((vals**2).sum())
        #vals = vals / vals_norm
        values[dx,:] = vals
    _,svs,_ = np.linalg.svd(values)
    #svs[0] is the largest singular value                      
    values = values / svs[0]
    return values

In [ ]:
# parameters of the rnn as shared variables
def get_parameter(n_in, n_out, n_hid):
    b_h = theano.shared(np.zeros(n_hid, dtype=dtype)) 
    h0 = theano.shared(np.zeros(n_hid, dtype=dtype))
    W_ih = theano.shared(sample_weights(n_in, n_hid))
    W_hh = theano.shared(sample_weights(n_hid, n_hid))
    W_ho = theano.shared(sample_weights(n_hid, n_out))
    b_o = theano.shared(np.zeros(n_out, dtype=dtype)) 
    return W_ih, W_hh, b_h, W_ho, b_o, h0

W_ih, W_hh, b_h, W_ho, b_o, h0 = get_parameter(n_in, n_out, n_hid)   
params = [W_ih, W_hh, b_h, W_ho, b_o, h0]

In [ ]:
# we could use the fact that maximaly two outputs are on, 
# but for simplicity we assume independent outputs:
def logistic_function(x):
    return 1./(1 + T.exp(-x))

# sequences: x_t
# prior results: h_tm1
# non-sequences: W_ih, W_hh, W_ho, b_h
def one_step(x_t, h_tm1, W_ih, W_hh, b_h, W_ho, b_o):
    h_t = T.tanh(theano.dot(x_t, W_ih) + theano.dot(h_tm1, W_hh) + b_h)
    y_t = theano.dot(h_t, W_ho) + b_o 
    y_t = logistic_function(y_t) 
    return [h_t, y_t]

In [ ]:
# hidden and outputs of the entire sequence
[h_vals, o_vals], _ = theano.scan(fn=one_step, 
                                  sequences = dict(input=v, taps=[0]), 
                                  outputs_info = [h0, None], # corresponds to return type of fn
                                  non_sequences = [W_ih, W_hh, b_h, W_ho, b_o] )

In [ ]:
# target values
target = T.matrix(dtype=dtype)

In [ ]:
# learning rate
lr = np.cast[dtype](0.2)
learning_rate = theano.shared(lr)

In [ ]:
cost = -T.mean(target * T.log(o_vals) + (1.- target) * T.log(1. - o_vals))

In [ ]:
def get_train_functions(cost, v, target):
    gparams = []
    for param in params:
        gparam = T.grad(cost, param)
        gparams.append(gparam)

    updates=[]
    for param, gparam in zip(params, gparams):
        updates.append((param, param - gparam * learning_rate))
    learn_rnn_fn = theano.function(inputs = [v, target],
                                   outputs = cost,
                                   updates = updates)
    return learn_rnn_fn

In [ ]:
learn_rnn_fn = get_train_functions(cost, v, target)